#  📌 하이브리드 추천 (SVD + TF-IDF)

## 1. 한 번만 실행

### ✏️ DB 연결

In [193]:
import dotenv
import os
from pymongo import MongoClient

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)

db = client.animation
dbcol_detail = db.ani_info
dbcol_review = db.review
dbcol_feat = db.ani_feature

### ✏️ import

In [194]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### ✏️ DataFrame 가져오고 columns 수정

In [195]:
# 애니 정보 불러오기
ani_df = pd.DataFrame(dbcol_detail.find({},{"id":1, "name":1, "series_id":1}))
ani_df.drop('_id', axis = 1, inplace = True)
ani_df.columns = ["ani_id", "ani_name", "series_id"]

# 평가 데이터 불러오기
profile = list(dbcol_review.find({}))
rating_df = pd.DataFrame(profile)
rating_df = rating_df[["profile", "animation", "score"]]
rating_df.columns = ["user_id", "ani_id", "score"]
# rating_df = rating_df.sort_values(by = ["user_id", "ani_id"])

# 형태소 분석이 이뤄진 데이터 불러오기
feat_df = pd.DataFrame(dbcol_feat.find({},{"id":1, "feat_str":1}))

### ✏️ Matrix로 변경

In [196]:
# 사용자 - 애니 pivot table 생성
user_ani_ratings_df = rating_df.pivot(
    index='user_id',
    columns='ani_id',
    values='score'
).fillna(0)

### ✏️ id, idx, series, name, feat을 통해 다른 것을 찾는 코드들

In [197]:
ani_df_id_to_idx = dict(zip(ani_df["ani_id"], ani_df.index))
ani_df_idx_to_series = dict(zip(ani_df.index, ani_df["series_id"]))
ani_df_id_to_series = dict(zip(ani_df["ani_id"], ani_df["series_id"]))
ani_df_id_to_name = dict(zip(ani_df["ani_id"], ani_df["ani_name"]))
feat_df_id_to_idx = dict(zip(feat_df["id"], feat_df.index))
feat_df_idx_to_feat = dict(zip(feat_df.index, feat_df["feat_str"]))

### ✏️ SVD를 활용한 애니메이션 30개 추천 함수

In [200]:
def recommend_ani(df_svd_preds, userId, ori_ani_df, ori_score_df, n):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 애니 데이터 정렬 -> 애니 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[userId].sort_values(ascending=False)
    print(sorted_user_predictions)
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_score_df[ori_score_df.user_id == userId]
    
    # 위에서 뽑은 user_data와 원본 애니 데이터를 합친다. 
    user_history = user_data.merge(ori_ani_df, on = 'ani_id').sort_values(['score'], ascending=False)
    
    # 유저가 이미 본 애니 id를 뽑는다. (TF-IDF에 사용)
    user_history_list = user_history['ani_id'].values.tolist()
    
    # 유저가 이미 본 애니 series id를 뽑는다.
    user_history_series_list_all = user_history['series_id'].values.tolist()
    user_history_series_list = []
    for id in user_history_series_list_all:
        # 시리즈가 미리 추가되어 있지 않고 nan이 아니면 히스토리 시리즈 리스트에 추가
        if id not in user_history_series_list and np.isnan(id) == False:
            user_history_series_list.append(id)
    
    # 원본 애니 데이터에서 사용자가 본 애니 데이터를 제외한 데이터를 추출
    recommendations = ori_ani_df[~ori_ani_df['ani_id'].isin(user_history['ani_id'])]
    
    # 사용자의 애니 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'ani_id')
    
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendation_list_all = list(recommendations.sort_values(userId, ascending = False)['ani_id'])

    # 추천 애니메이션 저장 리스트
    recommendation_list = []
    
    # 시리즈 저장 리스트
    series_list = user_history_series_list[:]
    
    for id in recommendation_list_all:
        
        # 선택한 애니메이션의 인덱스 가져옴
        idx = ani_df_id_to_idx[id]

        # 해당 인덱스의 시리즈 아이디 가져옴
        series = ani_df_idx_to_series[idx]
        
        # 시리즈가 미리 추가되어 있지 않다면 추천결과에 추가
        if series not in series_list:
            recommendation_list.append(id)
            # nan이 아니면 시리즈 리스트에 추가
            if np.isnan(series) == False:
                series_list.append(series)
        
        # 30개가 추천되면 종료
        if len(recommendation_list) == n:
            return user_history_list, recommendation_list

### ✏️ TF-IDF를 활용한 애니메이션 30->14개 추천 함수

In [208]:
def recommend_ani_2(user_history_list, recommendation_list, n):
    tf_idf_list = list()
    user_feat = ''
    for i in user_history_list:
        idx = feat_df_id_to_idx[i]
        feat = feat_df_idx_to_feat[idx]
        user_feat += feat
        user_feat += ' '
    tf_idf_list.append(user_feat)
    for j in recommendation_list:
        idx = feat_df_id_to_idx[j]
        reco_feat = feat_df_idx_to_feat[idx]
        tf_idf_list.append(reco_feat)
    
    tf_idf = TfidfVectorizer()
    tf_idf_matrix = tf_idf.fit_transform(tf_idf_list)
    cosine_sim = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n+1)]

    id_list = list()
    for num in range(n):
        id_list.append(recommendation_list[sim_scores[num][0]-1])
    
    return id_list

## 2. 유저가 리뷰를 남길 때마다 실행

#### 1. 리뷰를 남기면 user_id, ani_id, score를 가지고 옴

#### 2. DB와 rating_df에 데이터 추가

#### 3. user_ani_rating_df의 index에 user_id가 없으면 0으로 이뤄진 행 추가 

#### 4. 행(user_id), 열(ani_id)에 값을 0점에서 값(score)로 업데이트

In [202]:
# 리뷰를 남기면 불러오기
new_data = {
    "user_id" : 6000001,
    "ani_id" : 164,
    "score" : 4.0,
    
#     "user_id" : 6000002,
#     "ani_id" : 9450,
#     "score" : 4.0,
    
#     "user_id" : 6000002,
#     "ani_id" : 41053,
#     "score" : 4.0,
}

# DB에 추가
dbcol_review.insert_one({"profile": new_data["user_id"], "score": new_data["score"], "animation": new_data["ani_id"]})

# rating_df에 데이터 추가
rating_data = [[new_data["user_id"], new_data["ani_id"], new_data["score"]]]
temp_col = ['user_id', 'ani_id', 'score']
temp1 = pd.DataFrame(data=rating_data, columns=temp_col)
rating_df = pd.concat([rating_df, temp1], ignore_index=True)

# 리뷰를 처음 남기는 사람이라면 0으로 이뤄진 행 추가
if new_data["user_id"] not in user_ani_ratings_df.index:
    temp = [0 for i in range(4586)]
    temp2 = pd.DataFrame([temp], index=[new_data["user_id"]], columns = user_ani_ratings_df.columns)
    temp2.index.names = ["user_id"]
    user_ani_ratings_df = pd.concat([user_ani_ratings_df, temp2])

# 0점을 score값으로 변경
user_ani_ratings_df.loc[new_data["user_id"], new_data["ani_id"]] = new_data["score"]

In [203]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = user_ani_ratings_df.values

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-애니에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

# U 행렬, sigma 행렬, V 전치 행렬을 반환.
U, sigma, Vt = svds(matrix_user_mean, k = 12)

# 위는 0이 아닌 값만 포함되었기에 0이 포함된 대칭행렬로 diag를 이용해 변환
sigma = np.diag(sigma)

# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 아까 평균을 빼었으니 다시 사용자 평균 rating 더한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, index = user_ani_ratings_df.index, columns = user_ani_ratings_df.columns)

In [204]:
# pd.DataFrame(matrix_user_mean, index = user_ani_ratings_df.index, columns = user_ani_ratings_df.columns)
# df_svd_preds
# svd_user_predicted_ratings

In [205]:
# input_user_id = 4523846
# input_user_id = 4810880
# input_user_id = 6000001
input_user_id = new_data["user_id"]

user_history_list, recommendation_list = recommend_ani(df_svd_preds, input_user_id, ani_df, rating_df, 28)

# 최종 추천 애니 id 리스트
id_list = recommend_ani_2(user_history_list, recommendation_list, 14)

ani_id
24217    0.003099
23661    0.002654
40655    0.002178
13494    0.001920
13176    0.001824
           ...   
40520   -0.002804
40156   -0.002907
39992   -0.003126
40157   -0.004041
40110   -0.004082
Name: 6000001, Length: 4586, dtype: float64


In [206]:
print("##### 내가 본 애니!")
for result in user_history_list:
    print(ani_df_id_to_name[result])
print(" ")
print("##### 추천받은 애니!")
for result in id_list:
    print(ani_df_id_to_name[result])

##### 내가 본 애니!
라바 시즌 1
 
##### 추천받은 애니!
(더빙) 디지몬 어드벤처
천원돌파 그렌라간
네모바지 스폰지밥
극장판 짱구는 못말려 22기 : 정면승부! 로봇아빠의 역습
코드기아스 반역의 를르슈 R2
PSYCHO-PASS 1기
(자막) 초속 5센티미터
(더빙) 원피스 1기
그 비스크 돌은 사랑을 한다
바케모노가타리
암살교실 1기
강철의 연금술사 오리지널
4월은 너의 거짓말
(더빙) 강철의 연금술사 BROTHERHOOD


In [71]:
# rating_df
# user_ani_ratings_df
# dbcol_review.find_one({"profile": new_data["user_id"]})
# dbcol_review.insert_one({"profile": new_data["user_id"], "score": new_data["score"], "animation": new_data["ani_id"]})
# dbcol_review.delete_one({"profile": new_data["user_id"]})

### 정확도 (~ing(인터넷 이슈..))
기존 추천은 시리즈를 고려하여 추천하기에 올바른 테스트가 진행되지 않음.  
추천하는 과정에서 같은 시리즈면 추천하지 않는다는 부분을 제외하고 진행해볼 예정.
정확도 이전까지가 거진 확정이라 할 수 있으며 변경사항은 30, 14와 같은 숫자를 지정해뒀던 것을 n이라는 숫자를 넣었다는 것과 30 대신 28로 하였다는 것 정도.
정확도를 하기 위해서 추천 함수를 손대어야 하는데 Test를 위함이기에 복사해서 따로 함수를 새로 만들어서 테스트 할 예정.
그리고 테스트 후 파일 분리하여 push할 예정.

In [229]:
# user_id로 그룹화해서 count
rating_group = rating_df.groupby("user_id").count()
# user_id를 기준으로 ani_id, score가 count되어서 합쳐졌기에 10개 이상인 것의 user_id만 list로
group_list = rating_group[rating_group["ani_id"] >= 10].index.tolist()
# rating_df에서 위의 리스트에 속하는 것들만 따로 추출
rating_filter_df = rating_df[rating_df["user_id"].isin(group_list)]
# 추출한 것에서 for문으로 하나씩 테스트

# user_id에 맞는 행들만 따로 추출
rating_userid_df = rating_filter_df.groupby("user_id").get_group(group_list[0])
# 20%를 test하기 위한 데이터로 추출
test = rating_userid_df.sample(frac=0.1)
# 결과 예측할 때 사용하기 위한 n
test_n = len(test)
# 이 데이터의 ani_id만 list로
test_ani_id = list(test["ani_id"])
# user_id가 일치하고 ani_id가 test_ani_id에 속하는 것의 index만 추출
test_idx_list = list(rating_df[(rating_df["user_id"] == group_list[0]) & (rating_df["ani_id"].isin(test_ani_id))].index)
# rating_df를 복사
rating_df_temp = rating_df.copy()
# rating_df에서 본 것을 삭제
rating_df_temp = rating_df_temp.drop(test_idx_list)
# user_ani_ratings_df를 복사
user_ani_ratings_df_temp = user_ani_ratings_df.copy()
# score값을 0점으로 변경
for t in test_ani_id:
    user_ani_ratings_df_temp.loc[group_list[0], t] = 0

# SVD
matrix = user_ani_ratings_df_temp.values
user_ratings_mean = np.mean(matrix, axis = 1)
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)
U, sigma, Vt = svds(matrix_user_mean, k = 12)
sigma = np.diag(sigma)
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, index = user_ani_ratings_df_temp.index, columns = user_ani_ratings_df_temp.columns)

input_user_id = group_list[0]
user_history_list, recommendation_list = recommend_ani(df_svd_preds, input_user_id, ani_df, rating_df_temp, (test_n)*4)
# 최종 추천 애니 id 리스트
id_list = recommend_ani_2(user_history_list, recommendation_list, (test_n)*4)

ani_id
40562    5.220644
40815    4.985821
40372    1.383859
40825    1.285888
39431    1.107148
           ...   
40520   -0.267890
25004   -0.351012
24217   -0.459784
39986   -0.622442
23661   -0.812886
Name: 23, Length: 4586, dtype: float64


[40510,
 39081,
 39745,
 40261,
 40655,
 39985,
 39992,
 39648,
 39996,
 40790,
 40794,
 39856,
 40630,
 40159,
 40532,
 40829]

In [230]:
# id_list
# test_ani_id

[40655, 40681, 39072, 40615]

In [231]:
for t in test_ani_id:
    if t in id_list:
        print(t)

40655


In [228]:
print("##### 내가 본 애니!")
for result in user_history_list:
    print(ani_df_id_to_name[result])
print(" ")
print("##### 추천받은 애니!")
for result in id_list:
    print(ani_df_id_to_name[result])
print(" ")
print("##### Test로 뺀 애니!")
for result in test_ani_id:
    print(ani_df_id_to_name[result])

##### 내가 본 애니!
귀멸의 칼날 : 환락의 거리편
샤를로트
라이플 이즈 뷰티풀
평온세대의 위타천들
마나리아 프렌즈
논논비요리 : 논스톱
괴인 개발부의 쿠로이츠 씨
히나마츠리
감옥학원
아케비의 세일러복
나를 좋아하는 건 너뿐이냐
유루캠△ 1기 - 판권 부활
이 게임 폐인이 사는 법
(자막) 스파이 패밀리 part 1
쟈히님은 기죽지 않아!
(자막, 무삭제) 이 멋진 세계에 축복을! 2기
최애가 부도칸에 가 준다면 난 죽어도 좋아
리아데일의 대지에서
그 비스크 돌은 사랑을 한다
카구야 님은 고백받고 싶어 ~천재들의 연애 두뇌전~ 1기
코바야시네 메이드래곤 S
세계 최고의 암살자, 이세계 귀족으로 전생하다
이세계 미소녀 수육 아저씨와
귀멸의 칼날 : 무한열차편
다가시카시 2기
카구야 님은 고백받고 싶어 ~천재들의 연애 두뇌전~ 2기
하코즈메 ~파출소 여자의 역습~
현실주의 용사의 왕국 재건기 2기
흔해빠진 직업으로 세계최강 2기
뻐꾸기 커플
마왕학원의 부적합자
종말의 하렘
게이머즈!
흔해빠진 직업으로 세계최강 1기
달링 인 더 프랑키스
현자의 제자를 자칭하는 현자
현실주의 용사의 왕국 재건기 1기
 
##### 추천받은 애니!
무직전생 : 이세계에 갔으면 최선을 다한다 2기
방패 용사 성공담 Season 1
마왕성에서 잘 자요
어쨌든 귀여워
아하렌 양은 알 수가 없어
귀엽기만 한 게 아닌 시키모리 양
장난을 잘 치는 타카기 양 3기
천재 왕자의 적자국가 재생술
 
##### Test로 뺀 애니!
장난을 잘 치는 타카기 양 3기
조난입니까?
(무삭제) 이종족 리뷰어스
전생했더니 슬라임이었던 건에 대하여 2기 1부
